In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ChatBot/rcoe22-sem5-group3


In [ ]:
!pip install transformers;
!pip install flask-ngrok;
!pip install flask-bootstrap;
!pip install pyngrok;
# !ngrok authtoken pass

In [3]:
!ngrok authtoken <your Token>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
from flask import session,redirect, flash 
import json
import chatbuddy

import torch
import pickle
open_file = open("model.pickle", "rb")
model= pickle.load(open_file)
open_file.close()
open_file = open("tokenizer.pickle", "rb")
tokenizer= pickle.load(open_file)
open_file.close()

def question_answer(question, text):
    #tokenizes the question and the text given as a pair
    input_ids = tokenizer.encode(question, text)
    #converted to string inn the form of cls and sep
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    #segment IDs of the inputs
    #when was sep token first used in the list of the tokenized version
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    #no of tokens in Question
    num_seg_a = sep_idx+1
    #no of tokens in text given
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s for segment embeddings
    #1 for tokens that are not masked,0 for tokens that are masked.
    #Bert masks 15% percent of the sentence randomly hence....
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids, token type ids define the first or second portion of the input 
    output = model(torch.tensor([input_ids]),
                   token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer with start and end logits
    #start logits are span start scores, ends are vice versa
    answer_start_index = output.start_logits.argmax()
    answer_end_index = output.end_logits.argmax()
    
    #forming of the answers
    if answer_end_index >= answer_start_index:
        answer = tokens[answer_start_index]
        for i in range(answer_start_index+1, answer_end_index+1):
            if tokens[i][0:2] == "##": # here ## stands as a subword for example if the word is tokenization --> token and ##ization
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    # if answer starts with cls.....
    else:
        answer = "Unable to find the answer to your question."
    return answer.capitalize()


with open("config.json","r") as c:
    params=json.load(c)["params"]

#this is just an extention since we were getting 1 input from the form in the files
def chatbot_response(usertext):
    with open("text.txt","r") as f:
        data=f.read()
    s=question_answer(usertext,data)
    return s.capitalize()+"."

#this is chatbuddy section
def chatbuddy_response(usertext):
    s=chatbuddy.predict(usertext)
    return s

app = Flask(__name__)
run_with_ngrok(app)
app.secret_key="super-secret-key"

#this is for index page
@app.route("/")
def home():
    return render_template('index.html',params=params)

#this is the flask for textchat aka chatbot 
@app.route("/textchat")
def textchat():
    return render_template('textchat.html')
#this is for getting the inputs from the chatbot and running the same through the model 
@app.route("/textchat/get")
def get_bot_response():
    userText = request.args.get('msg')
    return chatbot_response(userText)



#for signup page aka dashboard
#this is the route for the dashboard page for editing and customizing the data
@app.route("/dashboard", methods=["GET","POST"])
def dashboard():
    if request.method=="POST":
            if "content" in request.form:
                with open("text.txt","w")as f:
                    f.write(request.form["content"])
                flash("Data Update Successfully","success")
            else:
                username = request.form.get("uname")
                userpass = request.form.get("pass")
                if username==params['admin_user'] and userpass==params['admin_password']:
                    # set the session variable
                    session['user']=username
                    with open("text.txt","r") as f:
                        data=f.read()
                        params["dataset"]=data
                    return render_template("dashboard.html", params=params)
                else:
                    flash("Username or Password Incorrect","danger")
                    return render_template("signin.html", params=params)

    if "user" in session and session['user']==params['admin_user']:
        with open("text.txt","r") as f:
            data=f.read()
            params["dataset"]=data
        return render_template("dashboard.html", params=params)
    else:
        return render_template("signin.html", params=params)


#this is for voicechat similarly functioning as that of textchat
@app.route("/voicechat")
def voice():
    return render_template("voicechat.html")
@app.route("/voicechat/get")
def get_bot_response2():
    userText = request.args.get('msg')
    return chatbot_response(userText)



#this is for logging out of the signin page 
@app.route('/logout')
def logout(): 
    if 'user' in session: 
        session.pop('user')
        return redirect('/dashboard')
    else:
        return render_template('index.html',params=params)

#chatbuddy section
@app.route('/chatbot')
def chatbot():
    return render_template('chatbot.html',params=params)
@app.route("/chatbot/get")
def get_bot_response3():
    userText = request.args.get('msg')
    return chatbuddy_response(userText)


app.run();

# if __name__ == '__main__':
  #  app.run()

